# data set-up

In [2]:
import numpy as np
import pandas as pd

# set numpy seed for reproducibility
np.random.seed(123)
# create a 1D numpy array "x1" with 10 normally distributed random numbers
x1 = np.random.randn(10)
x2 = np.random.randn(10)
x3 = np.random.randn(10)
# y is a 1D numpy array with 10 elements that are the weighted sum of x1, x2, and x3 with weights 0.6, 0.3, and 0.1 respectively
y = 0.6*x1 + 0.3*x2 + 0.1*x3

# y is a numpy array with 10 random binary 0/1 values
# y = np.random.randint(0, 2, size=10)

df = pd.DataFrame({'x1': x1, 'x2': x2, 'x3': x3, 'y': y})
df

,x1,x2,x3,y
0,-1.085631,-0.678886,0.737369,-0.781307
1,0.997345,-0.094709,1.490732,0.719068
2,0.282978,1.491390,-0.935834,0.523621
3,-1.506295,-0.638902,1.175829,-0.977865
4,-0.578600,-0.443982,-1.253881,-0.605743
5,1.651437,-0.434351,-0.637752,0.796781
6,-2.426679,2.205930,0.907105,-0.703518
7,-0.428913,2.186786,-1.428681,0.255820
8,1.265936,1.004054,-0.140069,1.046771
9,-0.866740,0.386186,-0.861755,-0.490364


# DecisionTreeClassifier

https://chatgpt.com/share/edd55b80-acb0-4d97-8534-ecb522d0a043


```
class DecisionTreeRegressor:
    def __init__(self, max_depth=None):
        # Initialize the decision tree regressor
        self.max_depth = max_depth  # Maximum depth the tree can grow to
        self.tree = None  # This will store the built tree structure

    def fit(self, X, y):
        # Fit the decision tree to the training data
        self.tree = self._build_tree(X, y)

    def predict(self, X):
        # Predict the target values for the input samples
        return np.array([self._predict_sample(sample, self.tree) for sample in X])

    def _mse(self, y):
        # Calculate the mean squared error of the target values
        if len(y) == 0:
            return 0  # To handle division by zero in case of empty split
        mean = np.mean(y)
        return np.mean((y - mean) ** 2)

    def _information_gain(self, y, y_left, y_right):
        # Calculate the reduction in mean squared error after a split
        parent_mse = self._mse(y)  # MSE of the parent node
        n = len(y)  # Total number of samples
        n_left, n_right = len(y_left), len(y_right)  # Number of samples in left and right nodes
        if n_left == 0 or n_right == 0:
            return 0  # If any split is empty, the information gain is zero
        # Weighted average MSE of the child nodes
        weighted_avg_mse = (n_left / n) * self._mse(y_left) + (n_right / n) * self._mse(y_right)
        # Information gain is the reduction in MSE
        return parent_mse - weighted_avg_mse

    def _best_split(self, X, y):
        # Find the best split for the data
        best_gain = 0  # Best information gain found
        best_split = None  # Best split feature and threshold
        best_left = None  # Best left split data
        best_right = None  # Best right split data

        n_samples, n_features = X.shape  # Number of samples and features
        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])  # Unique values of the feature
            for threshold in thresholds:
                left_indices = X[:, feature] <= threshold  # Indices of samples in left split
                right_indices = X[:, feature] > threshold  # Indices of samples in right split
                y_left, y_right = y[left_indices], y[right_indices]  # Target values of left and right splits
                gain = self._information_gain(y, y_left, y_right)  # Calculate information gain for this split
                if gain > best_gain:
                    best_gain = gain  # Update best gain
                    best_split = (feature, threshold)  # Update best split
                    best_left = (X[left_indices], y[left_indices])  # Update best left split data
                    best_right = (X[right_indices], y[right_indices])  # Update best right split data

        return best_gain, best_split, best_left, best_right

    def _build_tree(self, X, y, depth=0):
        # Recursively build the decision tree
        n_samples, n_features = X.shape  # Number of samples and features

        # Stop conditions for recursion
        if n_samples == 0 or (self.max_depth is not None and depth >= self.max_depth):
            # If no samples or max depth reached, create a leaf node
            leaf_value = np.mean(y)  # Mean of the target values in the node
            return {'leaf': True, 'value': leaf_value}

        # Find the best split
        gain, split, left, right = self._best_split(X, y)
        if gain == 0:
            # If no gain, create a leaf node with the mean of the target values
            leaf_value = np.mean(y)
            return {'leaf': True, 'value': leaf_value}

        # Create the decision node
        feature, threshold = split
        left_branch = self._build_tree(*left, depth + 1)  # Recursively build the left branch
        right_branch = self._build_tree(*right, depth + 1)  # Recursively build the right branch
        return {'leaf': False, 'feature': feature, 'threshold': threshold, 'left': left_branch, 'right': right_branch}

    def _predict_sample(self, sample, tree):
        # Predict the target value of a single sample by traversing the tree
        if tree['leaf']:
            return tree['value']  # If leaf node, return the value
        feature = tree['feature']
        threshold = tree['threshold']
        if sample[feature] <= threshold:
            return self._predict_sample(sample, tree['left'])  # Traverse left if feature value <= threshold
        else:
            return self._predict_sample(sample, tree['right'])  # Traverse right if feature value > threshold

# Example usage:
if __name__ == "__main__":
    # Toy dataset
    X = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])
    y = np.array([1.1, 1.9, 3.0, 3.9])

    # Create and train the decision tree regressor
    reg = DecisionTreeRegressor(max_depth=3)
    reg.fit(X, y)

    # Predict the target values for the same dataset
    predictions = reg.predict(X)
    print("Predictions:", predictions)
```

In [ ]:
class DecisionTreeRegressor():
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None # stores the tree structure

    def fit(self, X, y): # to the training data
        self.tree = self._build_tree(X, y) # private method

    def _mse(self, y): # mse of a leaf with mean being the average of all y target values in that leaf
        '''
        y = np.array([1.1, 1.9, 3.0, 3.9]) # y target values
        mean = np.mean(y)

        squared_differences = (y - mean) ** 2
        squared_differences = ([1.1, 1.9, 3.0, 3.9] - 2.475) ** 2
        squared_differences = [1.890625, 0.330625, 0.275625, 2.030625]

        mse = np.mean(squared_differences)
        '''
        leaf_prediction = np.mean(y)
        squared_differences = (y - leaf_prediction) ** 2
        mse_leaf = np.mean(squared_differences)

        return mse_leaf
    
    def _information_gain(self, y_left, y_right): # reduction in MSE after a new split by checking if the weighted MSE of both new child nodes is smaller than the MSE of the parent node
        n = len(y) # values in the parent node
        n_left, n_right = len(y_left), len(y_right) # observations in the child nodes
        if n_left == 0 or n_right == 0: # return 0 information gain if any child node is empty
            return 0
        
        weighted_avg_mse = (n_left / n) * self._mse(y_left) + (n_right / n) * self._mse(y_right)
        parent_mse = self._mse(y) # parent node, before the split
        information_gain = parent_mse - weighted_avg_mse
        return information_gain
    
    def _best_split(self, X, y):

        n_samples, n_features = X.shape # rows, columns
        for feature in range(n_features):
            thresholds = X